# Imports

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import sklearn
import tensorflow as tf
KERAS_BACKEND=tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
import gensim
# from keras_self_attention import SeqSelfAttention
# import keras
from keras.utils import np_utils
from keras.utils import to_categorical
# from keras.utils.vis_utils import model_to_dot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM
from gensim.models.keyedvectors import KeyedVectors

In [3]:
tf.__version__

'2.3.0'

In [4]:
 f = open("all_poses", "rb")
all_poses = pickle.load(f)
f.close()

f = open("flask_app_df", "rb")
poses_info = pickle.load(f)
f.close()

f = open("all_yoga_classes_df", "rb")
yoga_classes = pickle.load(f)
f.close()

f = open("poses_df", "rb")
class_poses = pickle.load(f)
f.close()

f = open("vinyasa_df", "rb")
more_vinyasa = pickle.load(f)
f.close()

f = open("hatha_df", "rb")
more_hatha = pickle.load(f)
f.close()

In [5]:
df = pd.concat([yoga_classes, more_vinyasa, more_hatha])
df = df.loc[df["Class Type"].isin(["Vinyasa", "Hatha", "Power", "Iyengar", "Ashtanga"])]
df.reset_index(inplace=True)

In [6]:
# average length of classes
lengths = []
for i in range(len(df["Poses"])):
    length = len(df["Poses"][i])
    lengths.append(length)
print(np.average(length))

33.0


# Word Embeddings

In [7]:
documents = [class_list for class_list in df["Poses"]]

In [59]:
embeddings = gensim.models.Word2Vec(documents, size=100, window=5, min_count=100, sg=1)

In [60]:
embeddings_size = len(list(embeddings.wv.vocab.items()))

In [61]:
embeddings_size

1918

In [62]:
embeddings.wv.index2word[:25]

['Downward Facing Dog Pose',
 'Mountain Pose',
 'Warrior Pose II',
 'Standing Forward Fold Pose',
 'Corpse Pose',
 'Child Pose',
 'Plank Pose',
 'Table Top Pose',
 'Cat Cow Pose',
 'High Lunge Pose',
 'Repeat Other Side',
 'Warrior Pose I',
 'Repeat ',
 'Easy Pose',
 'Three Legged Downward Facing Dog Pose',
 'Wind Release Pose',
 'Low Lunge Pose',
 'Chair Pose',
 'Bridge Pose',
 'Reverse Warrior Pose',
 'Triangle Pose',
 'Runners Lunge Pose',
 'Tree Pose',
 'Supine Spinal Twist Pose II',
 'Wide Child Pose']

In [63]:
print(embeddings.similarity("Corpse Pose", "Extended Side Angle Pose"))
print(embeddings.similarity("Corpse Pose", "Pigeon Pose"))
print(embeddings.similarity("Corpse Pose", "Child Pose"))
print(embeddings.similarity("Corpse Pose", "Wind Release Pose"))

0.11548326
0.21001613
0.29413182
0.733897


In [64]:
embeddings.most_similar("Corpse Pose", topn=20)

[('Corpse Pose (IRT)', 0.7811475992202759),
 ('Savasana Section', 0.7570167779922485),
 ('Corpse Pose Variation Bolster', 0.7357091903686523),
 ('Wind Release Pose', 0.7338969707489014),
 ('Corpse Pose Roll Under Spine', 0.7158660888671875),
 ('Corpse Pose Blanket', 0.7041505575180054),
 ('Constructive Rest Pose Arms Crossed', 0.6623439192771912),
 ('Constructive Rest Pose Hands In Anjali Mudra', 0.6598285436630249),
 ('Full Body Stretch Pose', 0.658948540687561),
 ('Supine Spinal Twist Pose II', 0.6543549299240112),
 ('Deep Relaxation Technique', 0.6541820168495178),
 ('Corpse Pose Variation Arms Up Toes', 0.6307822465896606),
 ('Happy Baby Pose', 0.630215048789978),
 ('Closing', 0.6282311677932739),
 ('Wind Release Pose Nose To Knee', 0.6265159249305725),
 ('Reclined Butterfly Pose Hands To Elbows', 0.6257239580154419),
 ('Reverse Pigeon Pose', 0.6203558444976807),
 ('Supine Windshield Wiper Pose Variation', 0.620124101638794),
 ('Wind Release Pose Exhale Inhale Flow', 0.616620302200

In [15]:
embedding_matrix = np.zeros((len(model.wv.vocab)+1, 100))
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [16]:
embedding_dim = embeddings.vector_size

# Neural Net

In [18]:
all_classes = [pose for yogaclass in documents for pose in yogaclass]

In [19]:
tokens = [pose for yogaclass in documents for pose in yogaclass]
print("Total tokens: ", len(tokens))
print("Unique tokens: ", len(set(tokens)))

Total tokens:  1906599
Unique tokens:  15372


In [42]:
length = 3 + 1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    sequences.append(seq)
print("Total Sequences:", len(sequences))

Total Sequences: 1906593


In [47]:
tokenizer = Tokenizer(num_words=embeddings_size)
tokenizer.fit_on_texts(sequences)
token_sequences = tokenizer.texts_to_sequences(sequences)
vocab_size = len(tokenizer.word_index) + 1

In [48]:
vocab_size

14675

In [45]:
X = np.asarray([np.asarray(x[:-1]) for x in token_sequences])
y = [y[-1] for y in token_sequences]
y = to_categorical(y, num_classes=vocab_size)
seq_length=len(X[1])

In [46]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
print(model.summary())

ValueError: Layer weight shape (14675, 100) not compatible with provided weight shape (2435, 100)